<a href="https://colab.research.google.com/github/Ditsuhi/ExploratoryAnalysis_FeatureSelection/blob/main/Traffic_Average_Speed_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import all required libraries
from arcpy import env
from collections import defaultdict
from  statistics import mean
import numpy
import time
from datetime import timedelta, date
start_time = time.time()

def makeFeatureLayerAndTableViewUsingArcpy (currentDate, hour):    
    TrafficStation = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\trafjanuary2019.shp"
    globals()[f"Traffic2019jan{currentDate.day}_{hour}"] = fr"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\2020dec31\2020dec31.gdb\sort_traf2019jan{currentDate.day}_{hour}_CopyRows" 
    arcpy.management.MakeFeatureLayer(TrafficStation, "TrafficStation")
    arcpy.MakeTableView_management(fr"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\2020dec31\2020dec31.gdb\sort_traf2019jan{currentDate.day}_{hour}_CopyRows", f"Traffic2019jan{currentDate.day}_{hour}")    
    return TrafficStation, f"Traffic2019jan{currentDate.day}_{hour}"


# to add  intensity,  occupancy time,  load, and traffic speed of files corresponding to all hours in a specific week
for date_dec in range(1, 8):
    for hour in range(0, 24):
        arcpy.CopyRows_management(fr"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\content\trafJan2019\sort_traf2019jan{date_dec}_{hour}.csv", fr"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\2020dec31\2020dec31.gdb\sort_traf2019jan{date_dec}_{hour}_CopyRows")
        end_date = date(2019, 1, date_dec)               
        print(str(end_date))
        TrafficStation, globals()[f"Traffic2019jan{end_date.day}_{hour}"] = makeFeatureLayerAndTableViewUsingArcpy(end_date, hour)
        inFeatures = TrafficStation
        joinField = "id"
        joinTable = globals()[f"Traffic2019jan{end_date.day}_{hour}"] 
        fieldList = ["intensidad", "ocupacion", "carga", "vmed"]
        arcpy.JoinField_management(inFeatures, joinField, joinTable, joinField, 
                                   fieldList)       
        

In [ ]:
# add new fields for calculating average intensity, average occupancy time, average load, and average speed

allColumns = ["avgIntens", "avgOcup", "avgCarga", "avgVmed"]

def addFieldsToFishnet (trafColumn):
    for col in trafColumn:
        arcpy.management.AddField(TrafficStation, col, "Double")
       
    
addFieldsToFishnet (allColumns) 

In [ ]:
#  extract all field's names

TrafficStation = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\trafjanuary2019.shp"
field_names = [f.name for f in arcpy.ListFields(TrafficStation)]

In [ ]:
# extract only those fields that we are interested in (average intensity, average occupancy time, average load, and average speed)
fields = field_names[12:]
print(fields)
# create update cursor for feature class 
with arcpy.da.UpdateCursor(TrafficStation, fields) as cursor:
    # calculate the average value of the corresponding fields
    for row in cursor:
        row[672] = mean([row[i] for i in range(0, 672, 4)])
        row[673] = mean([row[i] for i in range(1, 672, 4)])
        row[674] = mean([row[i] for i in range(2, 672, 4)])
        row[675] = mean([row[i] for i in range(3, 672, 4)])
        # update the cursor with the updated list
        cursor.updateRow(row)

# remove irrelevant fields
arcpy.management.DeleteField(TrafficStation, fields[: -4])